<a href="https://colab.research.google.com/github/ankitgoel1602/data-science/blob/master/data-analysis/pandas-groups/pandas_apply_user_defined_functions_on_groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import required libraries

In [ ]:
# upgrading Pandas
# !pip install -U pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import pandas, you should have Pandas version > 1.10.
import pandas as pd

# set max columns to display
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

# Dataset Details

The dataset which we are going to use here is the time-series data provided by <a href="https://data.worldbank.org/"> World Bank Open data </a> and is related to the crowd-sourced price data collected from 15 countries. For more details about the data, refer <a href="https://datacatalog.worldbank.org/dataset/crowdsourced-price-data-collection-pilot"> Crowdsourced Price Data Collection Pilot </a>.For this exercise, we are going to use data collected for Argentina.

In [ ]:
# uploading the file on google cloud.
# you can get the file from GITHUB and upload it here.
from google.colab import files
uploaded = files.upload()

In [ ]:
# importing data
initial_data = pd.read_csv('arg-crowdsourcedpdcpilot02_final_obs_all_clean.csv', parse_dates=['created_at'])

ParserError: Error tokenizing data. C error: Expected 1 fields in line 40, saw 23


In [ ]:
# top 5 rows
initial_data.head(5)

In [ ]:
# Lets select few columns for this exercise
columns_to_select = ['item_name','bh_name', 'brand', 'size', 'quantity', 'price', 'store_type', 'city_radius']

In [ ]:
data = initial_data[columns_to_select]

In [ ]:
# sample rows
data.head(5)

,item_name,bh_name,brand,size,quantity,price,store_type,city_radius
0,"Men's haircut, barber shop",Hairdressing salons and personal grooming esta...,NaN,1.0,1,50.00,private_service_provider,50
1,"Sardines, tinned, with skin, in vegetable oil,...",Preserved or processed fish and seafood,GV,160.0,1,29.30,small_medium_shop,20
2,"Broken rice, 25%, BNR",Rice,jumbo,1.0,1,12.49,NaN,50
3,"Men's shirt, WKB-M",Garments,george,1.0,1,349.00,small_medium_shop,20
4,"Vacuum cleaner, WKB-M",Major household appliances whether electric or...,philips,1.0,1,1799.00,small_medium_shop,50


# Getting categorical distribution per group.

To understand it better, we will go through few examples on how you can group the data and apply user defined functions to get distributions.

## Per category, what are the store types

In [ ]:
# Amount added in each hour
def get_distribution(column):
  column = column.fillna("None")
  return column.value_counts(dropna=False).to_dict()

def get_distribution_perc(column):
  column = column.fillna("None")
  return ((column.value_counts(dropna=False, normalize=True)*100).round(2)).to_dict()

In [ ]:
data.groupby('brand').agg(store_distribution = ('store_type',get_distribution_perc),
                          count_records = ('item_name','count')).reset_index().sort_values('count_records', ascending=False)

,brand,store_distribution,count_records
2728,tex,"{'small_medium_shop': 80.73, 'None': 15.1, 'specialized_shop': 4.17}",576
1059,carrefour,"{'small_medium_shop': 52.42, 'None': 34.3, 'specialized_shop': 13.29}",414
2153,none,"{'small_medium_shop': 63.52, 'None': 28.35, 'specialized_shop': 5.25, 'market': 2.36, 'bulk_discount_shop': 0.52}",381
1196,curitas,"{'small_medium_shop': 72.13, 'None': 17.76, 'specialized_shop': 8.47, 'market': 1.64}",366
2308,philips,"{'small_medium_shop': 48.29, 'None': 34.19, 'specialized_shop': 15.81, 'market': 1.71}",234
...,...,...,...
1285,donado,{'None': 100.0},1
1284,don valentin,{'small_medium_shop': 100.0},1
1283,don valentin,{'small_medium_shop': 100.0},1
1282,don satur,{'small_medium_shop': 100.0},1


## Aggregating data based on Week

In [ ]:
# total amount added each week
data.resample('W', on='created_at').price.sum().head(5)

created_at
2015-12-20     43056.38
2015-12-27     67338.51
2016-01-03     44434.59
2016-01-10     18222.36
2016-01-17    190838.54
Freq: W-SUN, Name: price, dtype: float64

In [ ]:
# By default, week starts with Sunday. Let's change it to start with Monday
data.resample('W-MON', on='created_at').price.sum().head(5)

created_at
2015-12-14     5532.86
2015-12-21    38507.62
2015-12-28    66863.29
2016-01-04    53924.10
2016-01-11    12608.69
Freq: W-MON, Name: price, dtype: float64

## Aggregating Data Based on Month

In [ ]:
data.resample('M', on='created_at').price.sum()

created_at
2015-12-31    1.538769e+05
2016-01-31    4.297143e+05
2016-02-29    9.352684e+05
2016-03-31    7.425185e+06
2016-04-30    1.384351e+07
2016-05-31    1.253785e+07
2016-06-30    7.264586e+06
2016-07-31    6.739416e+06
2016-08-31    2.721504e+06
Freq: M, Name: price, dtype: float64

In [ ]:
# by default month labels are assigned to the last day of month, lets change it
# to start from Month starting using 'MS' frequency
data.resample('MS', on='created_at').price.sum()

created_at
2015-12-01    1.538769e+05
2016-01-01    4.297143e+05
2016-02-01    9.352684e+05
2016-03-01    7.425185e+06
2016-04-01    1.384351e+07
2016-05-01    1.253785e+07
2016-06-01    7.264586e+06
2016-07-01    6.739416e+06
2016-08-01    2.721504e+06
Freq: MS, Name: price, dtype: float64

## Aggregations on different fields.

1. Unique items were added in each hour.
2. The total quantity was added in each hour.
3. The total amount was added in each hour.

In [ ]:
data.resample('H', on='created_at').agg({'price':'sum', 'quantity':'sum','item_code':'nunique'}).head(5)

,price,quantity,item_code
created_at,,,
2015-12-14 18:00:00,5449.90,3,3
2015-12-14 19:00:00,15.98,2,2
2015-12-14 20:00:00,66.98,7,4
2015-12-14 21:00:00,0.00,0,0
2015-12-14 22:00:00,0.00,0,0


# Grouping data based on different Time intervals

Here we will group on multiple fields along with time interval

## Amount added for each store type in each week.

In [ ]:
data.groupby([pd.Grouper(key='created_at', freq='W'), 'store_type']).price.sum().head(15)

created_at  store_type                
2015-12-20  other                          34300.00
            public_semi_public_service       833.90
            small_medium_shop               1206.04
2015-12-27  small_medium_shop                923.09
            specialized_shop               65164.00
2016-01-03  small_medium_shop                355.10
            specialized_shop               41922.00
2016-01-10  other                           8300.00
            small_medium_shop                412.55
            specialized_shop                3979.00
2016-01-17  market                           170.78
            other                         144341.00
            private_service_provider         220.00
            public_semi_public_service       271.84
            small_medium_shop              20908.29
Name: price, dtype: float64

## Amount added for each store type in each month.

In [ ]:
data.groupby([pd.Grouper(key='created_at', freq='M'), 'store_type']).price.sum().head(15)

created_at  store_type                
2015-12-31  other                          34300.00
            public_semi_public_service       833.90
            small_medium_shop               2484.23
            specialized_shop              107086.00
2016-01-31  market                           473.75
            other                         314741.00
            private_service_provider         325.00
            public_semi_public_service       276.79
            small_medium_shop              31042.79
            specialized_shop               29648.44
2016-02-29  market                          1974.04
            other                         527950.00
            private_service_provider        1620.00
            public_semi_public_service      1028.52
            small_medium_shop             224653.83
Name: price, dtype: float64

## Total Amount added based on item_name in each month.

In [ ]:
data.groupby([pd.Grouper(key='created_at', freq='M'), 'item_name']).price.sum()

created_at  item_name                                   
2015-12-31  Bar soap, solid, SB                                33.17
            Beer, domestic brand, single bottle, WKB           29.79
            Black tea, BL                                      12.00
            Black tea, in bags, WKB                            60.99
            Bread, white, sliced, WKB                          85.45
                                                              ...   
2016-08-31  Wheat flour, not self-rising, BL                  150.38
            White sugar, WKB                                  266.47
            Women's haircut, basic hairdresser               7730.00
            Wrist-watch, men's, CITIZEN Eco-Drive BM6060    52205.00
            Yoghurt, plain, WKB                               150.96
Name: price, Length: 1061, dtype: float64

## Multiple Aggregation based on store_type in each month.

In [ ]:
data.groupby([pd.Grouper(key='created_at', freq='M'), 'store_type'])\
    .agg(unique_items=('item_code', 'nunique'),
         total_quantity=('quantity','sum'),
         total_amount=('price','sum')).head(20)

unique_items  total_quantity  \
created_at store_type                                                 
2015-12-31 other                                  3               6   
           public_semi_public_service             1               1   
           small_medium_shop                     27              88   
           specialized_shop                       2              20   
2016-01-31 market                                 2              12   
           other                                  5              43   
           private_service_provider               2               2   
           public_semi_public_service             2               2   
           small_medium_shop                     73             472   
           specialized_shop                       7              24   
2016-02-29 market                                27              41   
           other                                  4              75   
           private_service_provider               5              11   
           public_semi_public_service             2               5   
           small_medium_shop                     77            1033   
           specialized_shop                      16              77   
2016-03-31 None                                   2              18   
           bulk_discount_shop                     2               3   
           market                                 5             101   
           other                                  6             487   

                                       total_amount  
created_at store_type                                
2015-12-31 other                           34300.00  
           public_semi_public_service        833.90  
           small_medium_shop                2484.23  
           specialized_shop               107086.00  
2016-01-31 market                            473.75  
           other                          314741.00  
           private_service_provider          325.00  
           public_semi_public_service        276.79  
           small_medium_shop               31042.79  
           specialized_shop                29648.44  
2016-02-29 market                           1974.04  
           other                          527950.00  
           private_service_provider         1620.00  
           public_semi_public_service       1028.52  
           small_medium_shop              224653.83  
           specialized_shop               117859.88  
2016-03-31 None                              164.20  
           bulk_discount_shop                222.90  
           market                           1807.68  
           other                         3355801.43